# Eye-Tracking Data Analysis for PLR Tests
This notebook implements analysis of Pupillary Light Reflex (PLR) tests using eye-tracking data.
The analysis includes data cleaning, pupil size extraction, signal quality assessment, and biomarker calculation.

In [ ]:
# Import required libraries

# pandas - For data manipulation and analysis of tabular data (landmarks and protocol files)
import pandas as pd

# numpy - For numerical operations and array manipulation of pupil measurements
import numpy as np

# matplotlib - For creating static visualizations of pupil response data
import matplotlib.pyplot as plt

# pathlib - For cross-platform file path handling
from pathlib import Path

# logging - For tracking program execution and debugging information
import logging

# scipy.signal - For signal processing operations like Savitzky-Golay filtering
from scipy import signal

# seaborn - For enhanced statistical data visualization built on matplotlib
import seaborn as sns

# shapely.geometry - For polygon area calculation
from shapely.geometry import Polygon

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

## Data Validating Functions
Functions to validate landmarks and protocol files

In [ ]:
def load_landmarks_file(file_path):
    """
    Load and validate landmarks data file
    
    Args:
        file_path (str): Path to landmarks CSV file
        
    Returns:
        pd.DataFrame: Processed landmarks data
    """
    try:
        df = pd.read_csv(file_path)
        
        # Required columns for timestamp and status
        base_columns = ['timestamp', 'id', 'retcode']
        
        # Generate expected landmark column names for both eyes
        landmark_columns = []
        for eye in ['left', 'right']:
            for i in range(1, 28):  # 27 landmarks per eye
                landmark_columns.extend([
                    f'{eye}_lm_{i}_x',
                    f'{eye}_lm_{i}_y'
                ])
        
        required_columns = base_columns + landmark_columns
        
        # Verify all required columns exist
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")
            
        logger.info(f"Successfully validated landmarks file {file_path.name} with {len(df)} rows")
        return df
    except Exception as e:
        logger.error(f"Error loading landmarks file: {e}")
        raise

In [ ]:
def load_protocol_file(file_path):
    """
    Load and validate protocol data file
    
    Args:
        file_path (str): Path to protocol CSV file
        
    Returns:
        pd.DataFrame: Protocol timing data
    """
    try:
        df = pd.read_csv(file_path)
        required_columns = ['time', 'event']
        if not all(col in df.columns for col in required_columns):
            raise ValueError("Missing required columns in protocol file")
        return df
    except Exception as e:
        logger.error(f"Error loading protocol file: {e}")
        raise

In [ ]:
data_path = Path().cwd().parent / "data"
for subfolder in data_path.iterdir():
    if subfolder.is_dir():
        landmarks_file = subfolder / f'{subfolder.name}_plr_landmarks.csv'
        protocol_file = subfolder / f'{subfolder.name}_plr_protocol.csv'
        
        # Load landmarks and protocol files
        landmarks_df = load_landmarks_file(landmarks_file)
        protocol_df = load_protocol_file(protocol_file)

## Data Cleaning
Functions to clean and preprocess the landmarks data

In [ ]:
def clean_landmarks_data(df):
    """
    Clean landmarks data by removing invalid frames
    
    Args:
        df (pd.DataFrame): Raw landmarks data
        
    Returns:
        pd.DataFrame: Cleaned landmarks data
    """
    # Remove frames with invalid retCode
    cleaned_df = df[df['retcode'] == 'OK'].copy()
    
    # Reset index after filtering
    cleaned_df.reset_index(drop=True, inplace=True)
    
    logger.info(f"Removed {len(df) - len(cleaned_df)} frames with invalid retCode")
    return cleaned_df

In [ ]:
# Find the landmarks.csv file
landmarks_file = list(data_path.glob('**/*landmarks.csv'))[0]

# Load landmarks data into DataFrame
landmarks_df = pd.read_csv(landmarks_file)

# Display first few rows
landmarks_df.head()

## Pupil Size Extraction
Functions to calculate pupil size and convert to millimeters

In [2]:
def calculate_pupil_size(landmarks_df):
    """
    Calculate pupil size using specified pupil landmarks
    
    Args:
        landmarks_df (pd.DataFrame): Landmarks data with one row
        
    Returns:
        tuple: (left_pupil_area, right_pupil_area)
    """
    def get_pupil_polygon_points(eye: str) -> list[tuple[float, float]]:
        """Get ordered polygon points for pupil outline"""
        # Order matters for correct polygon formation
        landmark_sequence = [7, 25, 9, 22, 10, 23, 24, 7]  # Close the polygon by repeating the first point
        
        points = []
        for lm in landmark_sequence:
            x = landmarks_df[f'{eye}_lm_{lm}_x'].iloc[0]
            y = landmarks_df[f'{eye}_lm_{lm}_y'].iloc[0]
            points.append((x, y))
        
        return points
    
    # Calculate areas for both eyes
    left_points = get_pupil_polygon_points('left')
    right_points = get_pupil_polygon_points('right')
    
    # Create polygons and calculate areas
    left_polygon = Polygon(left_points)
    right_polygon = Polygon(right_points)
    
    return left_polygon.area, right_polygon.area

In [3]:
def calculate_pupil_size_with_interpolation(landmarks_df):
    """
    Calculate pupil size using interpolation for a smoother boundary
    
    Args:
        landmarks_df (pd.DataFrame): Landmarks data with one row
        
    Returns:
        tuple: (left_pupil_area, right_pupil_area)
    """
    def get_pupil_center_and_points(eye: str):
        # Get center (landmark 8)
        center_x = landmarks_df[f'{eye}_lm_8_x'].iloc[0]
        center_y = landmarks_df[f'{eye}_lm_8_y'].iloc[0]
        center = (center_x, center_y)
        
        # Get known boundary points - using all the relevant pupil landmarks
        boundary_points = []
        for lm in [7, 25, 9, 22, 10, 23, 24]:  # Include all pupil landmarks
            x = landmarks_df[f'{eye}_lm_{lm}_x'].iloc[0]
            y = landmarks_df[f'{eye}_lm_{lm}_y'].iloc[0]
            boundary_points.append((x, y))
            
        return center, boundary_points
    
    def interpolate_points(center, boundary_points, num_interpolated=36):
        """
        Interpolate points while ensuring all original boundary points are included
        
        Args:
            center: Tuple of (x, y) coordinates for the pupil center
            boundary_points: List of (x, y) coordinates for pupil boundary
            num_interpolated: Number of total points in the final polygon
            
        Returns:
            List of (x, y) coordinates forming a smooth polygon
        """
        # Convert to polar coordinates relative to center
        angles = []
        distances = []
        points_polar = []
        
        for p in boundary_points:
            dx = p[0] - center[0]
            dy = p[1] - center[1]
            angle = np.arctan2(dy, dx)
            distance = np.sqrt(dx**2 + dy**2)
            # Store the polar coordinates with the original point
            points_polar.append((angle, distance, p))
            
        # Sort by angle
        points_polar.sort(key=lambda pp: pp[0])
        
        # Extract the sorted original points and their angles
        original_angles = [pp[0] for pp in points_polar]
        original_distances = [pp[1] for pp in points_polar]
        original_points = [pp[2] for pp in points_polar]
        
        # Create angles for interpolation that include the original angles
        # First, wrap the angles and distances to handle the circular nature
        wrapped_angles = original_angles.copy()
        wrapped_distances = original_distances.copy()
        
        # If we don't cover the full circle, add the first point at the end with +2π
        if max(wrapped_angles) - min(wrapped_angles) < 1.9 * np.pi:
            wrapped_angles.append(wrapped_angles[0] + 2 * np.pi)
            wrapped_distances.append(wrapped_distances[0])
        
        # Create a cubic spline interpolation that will pass through the original points
        from scipy.interpolate import CubicSpline
        cs = CubicSpline(wrapped_angles, wrapped_distances, bc_type='periodic')
        
        # Generate interpolated angles, ensuring we include the original angles
        interp_angles = np.linspace(min(wrapped_angles), min(wrapped_angles) + 2*np.pi, num_interpolated, endpoint=False)
        
        # Get distances from spline
        interp_distances = cs(interp_angles)
        
        # Convert back to cartesian coordinates
        interp_points = []
        # Correcting the y-coordinate calculation during Cartesian conversion
        for angle, dist in zip(interp_angles, interp_distances):
            norm_angle = ((angle + np.pi) % (2 * np.pi)) - np.pi
            x = center[0] + dist * np.cos(norm_angle)
            y = center[1] + dist * np.sin(norm_angle)  # Fixed to use center[1] for y-coordinate
            interp_points.append((x, y))
        
        return interp_points
    
    # Process left eye
    left_center, left_boundary_points = get_pupil_center_and_points('left')
    left_interp_points = interpolate_points(left_center, left_boundary_points, num_interpolated=36)
    left_polygon = Polygon(left_interp_points)
    
    # Process right eye
    right_center, right_boundary_points = get_pupil_center_and_points('right')
    right_interp_points = interpolate_points(right_center, right_boundary_points, num_interpolated=36)
    right_polygon = Polygon(right_interp_points)
    
    return left_polygon.area, right_polygon.area

In [ ]:
# Compare regular and interpolated methods
first_frame = landmarks_df_cleaned.iloc[[0]]
regular_left, regular_right = calculate_pupil_size(first_frame)
interp_left, interp_right = calculate_pupil_size_with_interpolation(first_frame)

print(f'Regular method - Left: {regular_left:.2f}, Right: {regular_right:.2f} square pixels')
print(f'Interpolated method - Left: {interp_left:.2f}, Right: {interp_right:.2f} square pixels')